In [ ]:
!pip install langgraph langchain_core langchain_groq langchain_huggingface langchain_chroma langchain-tavily

In [ ]:
from typing import TypedDict,Literal,Annotated,List,Dict
from langgraph.graph import StateGraph,START,END
from langchain_groq import ChatGroq
import os
from google.colab import userdata,drive
drive.mount('/content/drive')
os.environ['GROQ_API_KEY']=userdata.get('GROQ_API_KEY')
os.environ['TAVILY_API_KEY']=userdata.get('TAVILLY_API_KEY')

from langgraph.graph.message import add_messages
from langgraph.prebuilt import  ToolNode
from pydantic import  BaseModel,Field
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.retrievers import MultiQueryRetriever
from langchain_core.runnables import RunnableLambda, RunnablePassthrough,RunnableParallel
from langchain.tools import tool
from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage
from langchain.prompts import PromptTemplate
from langchain_tavily import TavilySearch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from langchain_core.messages import BaseMessage


In [ ]:
class GraphState(BaseModel):
  messages:Annotated[List[BaseMessage],add_messages]
  final_output:str=None
  query_type:Literal['research','farewell']=None
  results:str=None
  input_query:str=None

def router(state:GraphState):

  class Output(BaseModel):
    query_type:Literal['research','farewell']

  llm=ChatGroq(model='llama-3.1-8b-instant').with_structured_output(Output)

  prompt=f"""You will be identifying type of user query .If query needs research respond with 'research'
            if query is farewell message(eg : Bye, Thanks, Ok ) respond with 'farewell'
            User Query -{state.messages[-1].content} """

  response=llm.invoke(prompt)
  print(response.query_type)
  return {'query_type':response.query_type}

def researcher(state:GraphState):

  user_query=state.messages[-1].content

  tavily_search_tool = TavilySearch(max_results=20,topic="finance",country='India')

  results_dict=tavily_search_tool.invoke(user_query)['results']

  results='/n/n'.join([content.get('content') for content  in results_dict])

  return {'results':results}

def summarizer (state:GraphState):

  results= state.results

  llm=ChatGroq(model='llama-3.1-8b-instant')

  prompt=f"""You are an summarizer, you will be receiving various informations from multiple websites which is raw and unclean data.
           Remove unwanted informations from results.
           Provide a precise and excellent summary in paragraph or points.

           User query - {state.messages[-1].content} .
           Results- {state.results}"""

  summary=llm.invoke(prompt)

  return  {'messages':summary.content,'input_query':state.messages[-1].content}

def convo_end(state:GraphState):

  return {'messages':
          'Goodbye Have a great time'}

graph=StateGraph(GraphState)

graph.add_node('router',router)
graph.add_node('researcher',researcher)
graph.add_node('summarizer',summarizer)
graph.add_node('convo_end',convo_end)


graph.set_entry_point('router')
graph.add_conditional_edges('router',
                            lambda x : x.query_type,
                            {'research':'researcher',
                             'farewell':'convo_end'})
graph.add_edge('convo_end',END)
graph.add_edge('researcher','summarizer')
graph.add_edge('summarizer',END)

app=graph.compile()

In [ ]:
answer= app.invoke({'messages':['What are best stocks for long term investment  for next 5 years in India?']})

research


In [ ]:
for m in answer['messages']:
  m.pretty_print()

================================ Human Message =================================

What are best stocks for long term investment  for next 5 years in India?
================================ Human Message =================================

**Summary: Best Stocks for Long-Term Investment in India for Next 5 Years**

Based on the analysis of various sources, including Forbes Advisor India, here are some of the best stocks for long-term investment in India for the next 5 years:

**Top Sectors to Invest in:**

1. **Electricals and Renewable Energy**: This sector has shown strong growth potential with relatively lower risk.
2. **Infrastructure**: Infrastructure projects are expected to drive growth in the coming years, making this sector a promising investment opportunity.
3. **Banking**: Banking sector is expected to see growth due to increasing demand for financial services.

**Best Stocks to Invest in:**

1. **HDFC Bank Limited (NYSE:HDB)**: One of the largest private sector banks in India